### Ayudantia

In [5]:
import os
from tqdm import tqdm 
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import numpy as np

import torch
import torchvision
from torch import nn
from torchinfo import summary
from torch.utils.data import Dataset, DataLoader

In [7]:
root = 'Custom-FashionMNIST'

img_path = []
img_class = []
#for every dir in root
for class_dir in os.listdir(root):
    #Do nothing when the dir name is .DS_Store
    if class_dir == '.DS_Store':
        continue
    #Path to dir in root
    class_dir_path = os.path.join(root, class_dir)
    #Get evry elemnts in the dir
    elems = [os.path.join(class_dir_path, x) for x in os.listdir(class_dir_path) if x != '.DS_Store']
    #Add image path
    img_path.extend(elems)
    
    if class_dir == 'other_labels':
        #Impares
        img_class.extend([int(x[-5]) for x in elems])
    else:
        #Pares
        #Change bag to 8 class
        if class_dir == 'bag':
            img_class.extend([8 for _ in range(len(elems))])
        #Dir name to int
        else:
            img_class.extend(int(class_dir) for _ in range(len(elems)))


In [8]:
import pandas as pd
pd.DataFrame({'paths': img_path, 'class': img_class})

,paths,class
0,Custom-FashionMNIST\0\image-id_1.jpg,0
1,Custom-FashionMNIST\0\image-id_10.jpg,0
2,Custom-FashionMNIST\0\image-id_10007.jpg,0
3,Custom-FashionMNIST\0\image-id_10027.jpg,0
4,Custom-FashionMNIST\0\image-id_1005.jpg,0
...,...,...
69995,Custom-FashionMNIST\other_labels\image-id_9990...,7
69996,Custom-FashionMNIST\other_labels\image-id_9991...,9
69997,Custom-FashionMNIST\other_labels\image-id_9992...,9
69998,Custom-FashionMNIST\other_labels\image-id_9995...,7


In [12]:
img_path_idxs = list(range(len(img_path)))
X_train_val, X_test, y_train_val, y_test = train_test_split(img_path_idxs,img_class, test_size=0.1, random_state=42, stratify=img_class)

X_train, X_val, y_train, y_val = train_test_split(X_train_val, y_train_val, test_size=0.2, random_state=42, stratify=y_train_val)

print(f'Cantidad de elementos en:\n  Train: {len(X_train)}\n  Val: {len(X_val)}\n  Test: {len(X_test)}')
print(f'\nIndices repetidos entre Train/Test: {len(set(X_test).intersection(set(X_train)))}')
print(f'\nIndices repetidos entre Train/Val: {len(set(X_train).intersection(set(X_val)))}')
print(f'\nIndices repetidos entre Test/Val: {len(set(X_test).intersection(set(X_val)))}')

Cantidad de elementos en:
  Train: 50400
  Val: 12600
  Test: 7000

Indices repetidos entre Train/Test: 0

Indices repetidos entre Train/Val: 0

Indices repetidos entre Test/Val: 0


Pytorch -> (Dataloaders (Carpetas)) -> Dataset Customs

In [13]:
class CustomImageDataset(Dataset):
    def __init__(self, img_path, img_class, img_idxs):
        self.split_indexes = img_idxs
        self.path = np.array(img_path)[img_idxs]
        self.labels = np.array(img_class)[img_idxs]
    
    def __len__(self):
        return len(self.split_indexes)
        
    def __getitem__(self, idx):
        image = torchvision.io.read_image(self.path[idx]).float()
        label = torch.tensor(self.labels[idx])
        return image, label

In [19]:
train_ds = CustomImageDataset(img_path, img_class, X_train)
valid_ds = CustomImageDataset(img_path,img_class, X_val)
test_ds = CustomImageDataset(img_path,img_class,X_test)



(tensor([[[  0.,   0.,   7.,   0.,   0.,   1.,   5.,   0.,  13.,   0.,   2.,
           110., 214., 210., 227., 215., 189.,   0.,  10.,   1.,   6.,   1.,
            16.,   0.,   0.,   0.,   0.,   0.],
          [  1.,   0.,   3.,   0.,   0.,   3.,   0.,   0.,   0., 149., 190.,
           207., 246., 137.,  97., 227., 219., 205., 133.,   5.,   0.,   3.,
             0.,   9.,   0.,   0.,   0.,   0.],
          [  4.,   0.,   0.,   0.,   2.,   6.,   0.,  11., 157., 230., 204.,
           184., 234., 187., 134., 231., 205., 183., 211., 182.,  11.,   6.,
            31.,   0.,   0.,   0.,   0.,   0.],
          [  4.,   0.,   0.,   0.,   1.,   6.,   0.,  45., 231., 197., 180.,
           165., 219., 247., 229., 234., 166., 191., 188., 219., 114.,   0.,
             0.,  17.,   0.,   0.,   0.,   0.],
          [  2.,   2.,   0.,   1.,   0.,   4.,   0.,  93., 198., 196., 208.,
           170., 219., 234., 229., 233., 164., 191., 193., 208., 146.,   1.,
             0.,   0.,   0.,   0.,   0

In [ ]:
if torch.cuda.is-av